In [1]:
import sys
import os
from datetime import datetime


# Adicionar o diretório raiz do projeto ao path (supondo que o diretório atual seja "notebooks" e "src" esteja no diretório pai)
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from src.data_processing.weather_processor import WeatherDataProcessor
from src.ml.rain_predictor import RainPredictor
from src.ml.temperature_predictor import TemperaturePredictor
from src.ml.city_predictor import CityTemperaturePredictor

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
from sklearn.linear_model import LogisticRegression

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/30 15:12:05 WARN Utils: Your hostname, NicholasM, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/07/30 15:12:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/nicholas/projects/IOT/venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/nicholas/.ivy2.5.2/cache
The jars for the packages stored in: /home/nicholas/.ivy2.5.2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-01fa2477-3b26-47ef-9d0e-150fe7039b2c;1.0
	confs: [default]
	found org.postgresql#postgresql;42.6.0 in central
	found org.checkerframework#checker-qual;3.31.0 in central
:: resolution report :: resolve 139ms :: artifacts dl 4ms
	:: modules in use:
	org.checkerframework#check

In [2]:
# Etapa 1: Processamento de dados

processor = WeatherDataProcessor()

# Carregar dados brutos
df_raw = processor.load_processed_data(source='iot_weather_db')

# Limpar dados
df_clean = processor.clean_data(df_raw)

# Feature engineering
df = processor.feature_engineering(df_clean)

print(f"Dataset completo: {len(df)} registros")
print(f"Período: {df['date'].min()} até {df['date'].max()}")

df.head()

Dataset completo: 145319 registros
Período: 2025-02-25 00:00:00 até 2025-07-18 00:00:00


,idcity,date,hour,temperature,feelslike,clouds,precipitation,humidity,pressure,wind_speed,day_of_week,month,season,dew_point,pressure_tendency,temp_humidity_interaction,wind_pressure_interaction,rain_flag
0,22,2025-02-28,7,291.61,290.20,0.0,0.0,26.33,1019.33,1.54,4,2,Summer,213.904342,0.00,7678.0913,1569.7682,0
1,29,2025-03-12,4,270.08,264.04,75.0,0.0,77.00,1005.00,5.59,2,3,Autumn,254.177760,-14.33,20796.1600,5617.9500,0
3,29,2025-03-03,12,277.08,273.44,75.0,0.0,85.33,999.00,4.46,0,3,Autumn,267.038684,-6.00,23643.2364,4455.5400,0
4,29,2025-03-06,12,281.29,278.04,0.0,0.0,78.75,998.75,6.04,3,3,Autumn,266.074879,-0.25,22151.5875,6032.4500,0
5,29,2025-03-11,17,272.04,266.12,75.0,0.0,71.80,1005.00,6.35,1,3,Autumn,251.899473,6.25,19532.4720,6381.7500,0


In [3]:
# ==============================================
# 3. REDUÇÃO DO DATASET PARA ACELERAR TREINAMENTO
# ==============================================

# Opção 1: Pegar apenas últimos N dias
last_days = 30
df_recent = df[df['date'] >= df['date'].max() - pd.Timedelta(days=last_days)].copy()
print(f"Últimos {last_days} dias: {len(df_recent)} registros")

# Opção 2: Amostragem aleatória (manter ordem temporal)
sample_size = 1000
if len(df) > sample_size:
    # Pegar índices espaçados uniformemente para manter sequência temporal
    step = len(df) // sample_size
    df_sampled = df.iloc[::step].copy()
    print(f"Amostra uniforme: {len(df_sampled)} registros")
else:
    df_sampled = df.copy()

# Opção 3: Agregação por período (ex: média de cada 6 horas)
df_aggregated = df.set_index('date').resample('6H').agg({
    'temperature': 'mean',
    'humidity': 'mean', 
    'pressure': 'mean',
    'wind_speed': 'mean',
    'precipitation': 'sum',
    'rain_flag': 'max'
}).dropna().reset_index()
print(f"Dados agregados (6h): {len(df_aggregated)} registros")

# Escolher dataset reduzido para usar no modelo
df_menor = df_recent  # Altere aqui para df_sampled ou df_aggregated conforme preferir

print(f"\nDataset escolhido para treinamento: {len(df_menor)} registros")
print(f"Memória estimada: {df_menor.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

Últimos 30 dias: 30072 registros
Amostra uniforme: 1003 registros
Dados agregados (6h): 141 registros

Dataset escolhido para treinamento: 30072 registros
Memória estimada: 5.2 MB


In [4]:
# # ==============================================
# # 4. Modelo de Previsão de Temperatura - Random Forest
# # ==============================================

# # Inicializar preditor de temperatura
# temp_predictor_rf = TemperaturePredictor(model_type='random_forest')

# # Treinar modelo
# print("=== TREINAMENTO RANDOM FOREST ===")
# metrics_rf = temp_predictor_rf.train_model(df_menor, target_column='temperature')

# # Exibir métricas
# print(f"\nMétricas Random Forest:")
# print(f"MAE: {metrics_rf['mae']:.2f}°C")
# print(f"RMSE: {metrics_rf['rmse']:.2f}°C") 
# print(f"R²: {metrics_rf['r2']:.4f}")
# print(f"CV MAE: {metrics_rf['cv_mae_mean']:.2f} ± {metrics_rf['cv_mae_std']:.2f}")

# # Importância das features
# importance_df = temp_predictor_rf.get_feature_importance()
# print(f"\nTop 10 Features mais importantes:")
# print(importance_df.head(10))

# # Visualizar importância
# plt.figure(figsize=(10, 6))
# plt.barh(importance_df.head(10)['feature'], importance_df.head(10)['importance'])
# plt.title('Importância das Features - Random Forest')
# plt.xlabel('Importância')
# plt.tight_layout()
# plt.show()

# # Fazer predições
# predictions_rf, _ = temp_predictor_rf.predict(df_menor.tail(50))
# print(f"\nPrimeiras 10 predições: {predictions_rf[:10]}")

In [5]:
# ==============================================
# 5. Modelo SARIMAX por Cidade - VERSÃO REFATORADA
# ==============================================

# Inicializar preditor
city_predictor = CityTemperaturePredictor(
    order=(2, 1, 1),
    seasonal_order=(1, 1, 1, 24),
    exog_lags=2
)
df_menor = df_menor[df_menor['idcity'] == 22]  # Filtrar por uma cidade específica para exemplo
print("=== TREINAMENTO DE MODELOS ===")
# Treinar modelos (função focada apenas no treinamento)
trained_cities = city_predictor.train_city_models(
    df_menor,
    target_column='temperature',
    city_column='idcity',
    exog_columns=['humidity', 'pressure', 'wind_speed'],
    test_size=0.2
)

print(f"Cidades treinadas: {trained_cities}")

print("\n=== AVALIAÇÃO DE MODELOS ===")
# Avaliar modelos (função dedicada às métricas)
city_metrics = city_predictor.evaluate_models()

# Exibir métricas
for city, metrics in city_metrics.items():
    print(f"\n🏙️ Cidade {city}:")
    print(f"   MAE: {metrics['mae']:.2f}°C")
    print(f"   RMSE: {metrics['rmse']:.2f}°C")
    print(f"   R²: {metrics['r2']:.4f}")
    print(f"   Modelo usado: {metrics['model_used']}")
    print(f"   Features exógenas: {metrics['exog_features']}")

print("\n=== DIAGNÓSTICOS DETALHADOS ===")
# Análise detalhada de uma cidade
cidade_exemplo = trained_cities[0]
diagnostics = city_predictor.get_model_diagnostics(cidade_exemplo)
print(f"AIC: {diagnostics['aic']:.2f}")
print(f"BIC: {diagnostics['bic']:.2f}")

print("\n=== PREVISÕES FUTURAS ===")
# Previsões futuras
for city in trained_cities:
    predictions, conf_int = city_predictor.predict_future(city, steps=6)
    print(f"\n🌡️ Cidade {city}:")
    for i, (pred, ci) in enumerate(zip(predictions, conf_int)):
        print(f"   Hora +{i+1}: {pred:.1f}°C (IC: {ci[0]:.1f} - {ci[1]:.1f})")

=== TREINAMENTO DE MODELOS ===
Cidades treinadas: [np.int32(22)]

=== AVALIAÇÃO DE MODELOS ===

🏙️ Cidade 22:
   MAE: 3.04°C
   RMSE: 3.83°C
   R²: -0.1338
   Modelo usado: with_exog
   Features exógenas: 12

=== DIAGNÓSTICOS DETALHADOS ===
AIC: 2165.99
BIC: 2241.41

=== PREVISÕES FUTURAS ===

🌡️ Cidade 22:
   Hora +1: 304.1°C (IC: 300.0 - 308.2)
   Hora +2: 303.7°C (IC: 299.5 - 307.9)
   Hora +3: 304.2°C (IC: 299.9 - 308.5)
   Hora +4: 303.7°C (IC: 299.4 - 308.0)
   Hora +5: 302.8°C (IC: 298.4 - 307.1)
   Hora +6: 302.9°C (IC: 298.6 - 307.2)
